In [1]:
# Import library yang dibutuhkan
import pandas as pd
import os
import joblib
from sklearn.metrics.pairwise import cosine_similarity
# import ast # Jika diperlukan untuk konversi list dari string

# --- Konfigurasi Path File ---
NAMA_FILE_PROCESSED_CSV = 'processed_data_v3.csv' # Menggunakan file V3
PATH_PROCESSED_DATA_DIR = os.path.join('..', 'data', 'processed')
PATH_PROCESSED_FILE = os.path.join(PATH_PROCESSED_DATA_DIR, NAMA_FILE_PROCESSED_CSV)

PATH_MODELS_DIR = os.path.join('..', 'models')
NAMA_FILE_TFIDF_VECTORIZER = 'tfidf_vectorizer.joblib'
PATH_TFIDF_VECTORIZER = os.path.join(PATH_MODELS_DIR, NAMA_FILE_TFIDF_VECTORIZER)

# Nama kolom yang relevan dari processed_df_v3
COLUMN_AIRCRAFT_TYPE = 'aircraft_type'
COLUMN_FINDING_DESC = 'finding_description'
# ... (dan nama kolom lainnya yang sudah kita definisikan)


# --- 1. Memuat Data yang Sudah Diproses dan TF-IDF Vectorizer ---
print(f"Mencoba memuat data dari: {PATH_PROCESSED_FILE}")
try:
    processed_df = pd.read_csv(PATH_PROCESSED_FILE)
    print("Data yang sudah diproses (v3) berhasil dimuat!")
    print(f"Kolom tersedia: {processed_df.columns.tolist()}")
except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan di {PATH_PROCESSED_FILE}")
    processed_df = pd.DataFrame()

print(f"\nMencoba memuat TF-IDF Vectorizer dari: {PATH_TFIDF_VECTORIZER}")
try:
    tfidf_vectorizer = joblib.load(PATH_TFIDF_VECTORIZER)
    print("TF-IDF Vectorizer berhasil dimuat!")
except FileNotFoundError:
    print(f"ERROR: File tidak ditemukan di {PATH_TFIDF_VECTORIZER}")
    tfidf_vectorizer = None

# --- 2. Fungsi untuk Mendapatkan Rekomendasi (DIPERBARUI DENGAN FILTER) ---
def get_recommendations_with_filter(new_finding_text, aircraft_type, top_n=3):
    """
    Memberikan rekomendasi berdasarkan kemiripan TF-IDF setelah memfilter berdasarkan tipe pesawat.
    """
    if not tfidf_vectorizer: return "TF-IDF Vectorizer tidak tersedia."
    if processed_df.empty: return "Data historis tidak tersedia."

    # 1. FILTER DATA BERDASARKAN TIPE PESAWAT
    # Pastikan perbandingan case-insensitive
    filtered_df = processed_df[processed_df[COLUMN_AIRCRAFT_TYPE].str.lower() == aircraft_type.lower()].reset_index(drop=True)
    
    if filtered_df.empty:
        return f"Tidak ada data historis untuk tipe pesawat '{aircraft_type}'."
    
    # 2. BUAT MATRIKS TF-IDF HANYA DARI DATA YANG SUDAH DIFILTER
    historical_tfidf_features = tfidf_vectorizer.transform(filtered_df[COLUMN_FINDING_DESC])

    # 3. TRANSFORMASI INPUT BARU & HITUNG KEMIRIPAN
    cleaned_new_finding_text = new_finding_text.lower()
    new_tfidf_features = tfidf_vectorizer.transform([cleaned_new_finding_text])
    similarities = cosine_similarity(new_tfidf_features, historical_tfidf_features)
    
    # 4. AMBIL REKOMENDASI TERATAS
    most_similar_indices = similarities[0].argsort()[-top_n:][::-1]

    recommendations_output = []
    for i, index in enumerate(most_similar_indices):
        similarity_score = similarities[0][index]
        if similarity_score > 0.01: # Batas minimal kemiripan
            # Ambil detail dari DataFrame yang sudah difilter
            recommendation_detail = filtered_df.iloc[index]
            recommendations_output.append({
                "rank": i + 1,
                "similarity_score": similarity_score,
                "aircraft_type": recommendation_detail.get(COLUMN_AIRCRAFT_TYPE),
                "historical_finding": recommendation_detail.get(COLUMN_FINDING_DESC),
                # ... (sertakan semua field lain seperti order_info, steps, man_hours, dll.)
                "rectification_steps": recommendation_detail.get('rectification_steps'),
                "man_hours_per_step": recommendation_detail.get('man_hours_per_step'),
                # ... (dst)
            })
    
    return recommendations_output

# --- 3. Contoh Penggunaan Fungsi Rekomendasi yang Baru ---
if not processed_df.empty and tfidf_vectorizer:
    # Tentukan input untuk pengujian
    contoh_finding_baru = "engine oil leak detected near section b" # Ganti dengan contoh Anda
    contoh_tipe_pesawat = "airbus a330" # Ganti dengan salah satu tipe pesawat yang ada di data Anda
    
    print(f"\n--- Contoh Penggunaan Rekomendasi dengan Filter ---")
    print(f"Input Finding: \"{contoh_finding_baru}\"")
    print(f"Filter Tipe Pesawat: \"{contoh_tipe_pesawat}\"")
    
    hasil_rekomendasi = get_recommendations_with_filter(contoh_finding_baru, contoh_tipe_pesawat, top_n=3)

    if isinstance(hasil_rekomendasi, str): 
        print(f"\nHASIL: {hasil_rekomendasi}")
    elif not hasil_rekomendasi:
        print("\nHASIL: Tidak ditemukan rekomendasi yang cukup mirip.")
    else:
        print("\n--- REKOMENDASI DITEMUKAN ---")
        for rec in hasil_rekomendasi:
            print(f"\nPeringkat: {rec.get('rank')}")
            print(f"Skor Kemiripan: {rec.get('similarity_score'):.4f}")
            print(f"Tipe Pesawat: {rec.get('aircraft_type')}")
            print(f"Finding Historis Mirip: {rec.get('historical_finding')}")
            print(f"Langkah Rektifikasi: {rec.get('rectification_steps')}")
            # ... (print field lainnya)
else:
    print("\nTidak bisa menjalankan contoh rekomendasi karena data atau vectorizer tidak siap.")

Mencoba memuat data dari: ..\data\processed\processed_data_v3.csv
Data yang sudah diproses (v3) berhasil dimuat!
Kolom tersedia: ['finding_description', 'aircraft_type', 'order_info', 'materials_info', 'rectification_steps', 'man_hours_per_step', 'work_centres_per_step', 'plants_per_step']

Mencoba memuat TF-IDF Vectorizer dari: ..\models\tfidf_vectorizer.joblib
TF-IDF Vectorizer berhasil dimuat!

--- Contoh Penggunaan Rekomendasi dengan Filter ---
Input Finding: "engine oil leak detected near section b"
Filter Tipe Pesawat: "airbus a330"

HASIL: Tidak ada data historis untuk tipe pesawat 'airbus a330'.
